In [133]:
#import packages and create date

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import datetime
from datetime import *
import shutil
import difflib
from pulp import *
import openpyxl
import glob

today = date.today()
# today = today - datetime.timedelta(days=2)
today = today.strftime("%m.%d.20%y")
today

'02.09.2022'

In [134]:
## FanDuel Player Pool

contest = "PlayerPool_"
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
contest_data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool'

fd = glob.glob('FanDuel-NBA*.csv')[0]
os.rename(fd, "FDSalaries_"+ contest + str(today)+".csv")

shutil.move(downloads + "/"+ "FDSalaries_"+ contest + str(today)+".csv",
           contest_data_loc + "/"+ "FDSalaries_"+ contest + str(today)+".csv")

os.chdir(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool")
fanduel_file = pd.read_csv(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool/FDSalaries_"+ contest + str(today)+".csv")
fanduel_file.to_excel(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool/FDSalaries_"+ contest + str(today)+".xlsx", index=False)
os.remove("FDSalaries_"+ contest + str(today)+".csv")


In [135]:
## ETR Daily

# changes the working directory to your downloads folder
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/ETR_Data/FanDuel'
daily_file = "FD NBA Projections.csv"

shutil.move(downloads + "/"+ daily_file,
           data_loc + "/"+ daily_file)

os.chdir(data_loc)

df = pd.read_csv(daily_file)

df.to_excel("ETR_Daily_"+str(today)+".xlsx", index=False)
os.remove("FD NBA Projections.csv")

In [136]:
fd = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\FD_Data\PlayerPool\FDSalaries_PlayerPool_' + str(today) + ".xlsx")
fd.head()

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15,Roster Position
0,71397-15755,PF/C,Anthony,Anthony Davis,Davis,48.244119,34,11200,LAL@POR,LAL,POR,GTD,Wrist,NaN,NaN,NaN,PF/C
1,71397-9488,SF/PF,LeBron,LeBron James,James,52.771054,38,10800,LAL@POR,LAL,POR,GTD,Knee,NaN,NaN,NaN,SF/PF
2,71397-67708,PG/SG,Dejounte,Dejounte Murray,Murray,48.206382,47,10600,SA@CLE,SA,CLE,NaN,NaN,NaN,NaN,NaN,PG/SG
3,71397-9714,SG/SF,DeMar,DeMar DeRozan,DeRozan,42.473999,50,10500,CHI@CHA,CHI,CHA,NaN,NaN,NaN,NaN,NaN,SG/SF
4,71397-60021,PF,Pascal,Pascal Siakam,Siakam,43.233333,39,10000,TOR@OKC,TOR,OKC,NaN,NaN,NaN,NaN,NaN,PF


In [137]:
## Joins the FD and ETR Files to include FD Positions 
    ## ETR does not include G/F/UTIL

os.chdir(r"C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis")
etr = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\ETR_Data\FanDuel\ETR_Daily_' + str(today) + ".xlsx")
fd = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\FD_Data\PlayerPool\FDSalaries_PlayerPool_' + str(today) + ".xlsx")

## TO DO - check if merge on only DKNG Roster Position Works
df = pd.merge(etr, 
              fd[['Nickname','Position', 'Id']], 
              left_on='Player', 
              right_on='Nickname')

In [138]:
df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id
0,LaMelo Ball,CHA,CHI,38.0,PG,"8,400",50.0,11.4,60.1,36.4,MAIN,LaMelo Ball,PG,71397-145485
1,Terry Rozier,CHA,CHI,40.0,SG/PG,"6,400",39.2,8.6,48.1,63.2,MAIN,Terry Rozier,SG/PG,71397-42677
2,Miles Bridges,CHA,CHI,40.0,SF,"7,200",41.8,8.0,50.5,39.6,MAIN,Miles Bridges,SF,71397-84678
3,Jusuf Nurkic,POR,LAL,33.0,C,"7,700",42.9,7.1,52.8,20.0,MAIN,Jusuf Nurkic,C,71397-55057
4,Justise Winslow,POR,LAL,32.0,SF,"4,300",28.2,6.0,37.2,26.2,MAIN,Justise Winslow,SF,71397-58084


In [139]:
# ## Unnests Roster Positions into each Row (explode in Python) ##


# ## Update Salary to INT 
df['FD Salary'] = df['FD Salary'].str.replace(",","").astype(float)

df['Roster Position'] = df['Position'].astype(str)
df['Roster Position'] = df['Position'].str.split('/')

df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id,Roster Position
0,LaMelo Ball,CHA,CHI,38.0,PG,8400.0,50.0,11.4,60.1,36.4,MAIN,LaMelo Ball,PG,71397-145485,[PG]
1,Terry Rozier,CHA,CHI,40.0,SG/PG,6400.0,39.2,8.6,48.1,63.2,MAIN,Terry Rozier,SG/PG,71397-42677,"[SG, PG]"
2,Miles Bridges,CHA,CHI,40.0,SF,7200.0,41.8,8.0,50.5,39.6,MAIN,Miles Bridges,SF,71397-84678,[SF]
3,Jusuf Nurkic,POR,LAL,33.0,C,7700.0,42.9,7.1,52.8,20.0,MAIN,Jusuf Nurkic,C,71397-55057,[C]
4,Justise Winslow,POR,LAL,32.0,SF,4300.0,28.2,6.0,37.2,26.2,MAIN,Justise Winslow,SF,71397-58084,[SF]


In [140]:
df.to_excel("FD_final_PlayerPool_ETR_"+str(today)+".xlsx", index=False)
# os.system("FD_final_PlayerPool_ETR_"+str(today)+".xlsx")

# Create the Constraint Problem

Goal: Maximize FD Points

- Total Players = 9
- TotalSalary <= 60000
- TotalPosition_PG
- TotalPosition_SG
- TotalPosition_SF
- TotalPosition_PF
- TotalPosition_C


In [141]:
prob = pulp.LpProblem('NBA_DFS_Daily', LpMaximize)

In [142]:
player_vars = [pulp.LpVariable(f'player_{row.Player}', cat='Binary') for row in df.itertuples()]

In [143]:
prob += pulp.lpSum(player_var for player_var in player_vars) == 9

In [144]:
# position constraints
def get_position_sum(player_vars, df, position):
    return pulp.lpSum([player_vars[i] * (position in df['FD Position'].iloc[i]) for i in range(len(df))])
    
# prob += get_position_sum(player_vars, df, 'PG') <= 3
prob += get_position_sum(player_vars, df, 'PG') >= 2
# prob += get_position_sum(player_vars, df, 'SG') <= 3
prob += get_position_sum(player_vars, df, 'SG') >= 2
# prob += get_position_sum(player_vars, df, 'G') <= 2
# prob += get_position_sum(player_vars, df, 'G') >= 2
# prob += get_position_sum(player_vars, df, 'SF') <= 2
prob += get_position_sum(player_vars, df, 'SF') >= 2
# prob += get_position_sum(player_vars, df, 'PF') <= 3
prob += get_position_sum(player_vars, df, 'PF') >= 2
# prob += get_position_sum(player_vars, df, 'F') <= 2
# prob += get_position_sum(player_vars, df, 'F') >= 2
# prob += get_position_sum(player_vars, df, 'C') <= 1
prob += get_position_sum(player_vars, df, 'C') >= 1
# prob += get_position_sum(player_vars, df, 'UTIL') >= 2
# prob += get_position_sum(player_vars, df, 'UTIL') <= 2



In [128]:
# total salary constraint

prob += pulp.lpSum(df['FD Salary'].iloc[i] * player_vars[i] for i in range(len(df))) <= 60000

In [129]:
# finally, specify the goal
prob += pulp.lpSum([df['FD Points'].iloc[i] * player_vars[i] for i in range(len(df))])

In [130]:
# solve and print the status
prob.solve()
print(LpStatus[prob.status])

Optimal


In [131]:
# for each of the player variables, 
total_salary_used = 0
mean_AvgPointsPerGame = 0
for i in range(len(df)):
    if player_vars[i].value() == 1:
        row = df.iloc[i]
        print(row['FD Position'], row.Player, row.Team, row['FD Salary'], row['FD Points'])
        total_salary_used += row['FD Salary']
        mean_AvgPointsPerGame += row['FD Points']
#mean_AvgPointsPerGame /= 9  # divide by total players in roster to get a mean
total_salary_used, mean_AvgPointsPerGame

SF Brandon Ingram NOP 7800.0 47.2
PG Keifer Sykes IND 3500.0 28.1
SF Lance Stephenson IND 4500.0 31.6
C Jonas Valanciunas NOP 7200.0 42.2
PG/SG Anfernee Simons POR 6500.0 36.5
PG Trae Young ATL 9000.0 46.3
PF Jaxson Hayes NOP 5500.0 31.2
PF/SF Giannis Antetokounmpo MIL 11300.0 54.2
SG Torrey Craig IND 4600.0 25.9


(59900.0, 343.2)